# Machine Learning with ibmdbpy and Db2 Warehouse: Association Rules Mining 

This notebook explains how to perform association rules mining with the ibmdbpy Python package and data stored in Db2 Warehouse.
___
### Accelerate Python analytics with in-database processing by using ibmdbpy and IBM Db2 Warehouse
 
The ibmdbpy project provides a Python interface for data manipulation and access to in-database algorithms in IBM Db2 Warehouse. It accelerates Python analytics by seamlessly pushing operations written in Python into the underlying database for execution, thereby benefitting from in-database performance-enhancing features, such as columnar storage and parallel processing. For more details about ibmdbpy, please refer to the [documentation](https://pythonhosted.org/ibmdbpy/index.html) and to the dedicated [Git repository](https://github.com/ibmdbanalytics/ibmdbpy/tree/master/ibmdbpy). This notebook provides you with an overview of ibmdbpy functionalities. 

 
__Prerequisites__
* Db2 account: see [IBM Cloud](https://cloud.ibm.com/login) or [Db2 Warehouse](https://www.ibm.com/support/knowledgecenter/en/SSCJDQ/com.ibm.swg.im.dashdb.kc.doc/welcome.html)
* Db2 driver: learn more on [IBM Knowledge Center](https://www.ibm.com/support/knowledgecenter/en/SSFMBX/com.ibm.swg.im.dashdb.doc/connecting/connect_applications_by_type.html) and see [IBM Support](https://www.ibm.com/support/pages/db2-jdbc-driver-versions-and-downloads)
* Having installed the [ibmdbpy](https://pypi.org/project/ibmdbpy/) python library with pip: 
> pip install ibmdbpy 
* Optional dependency for JDBC is the [jaydebeapi](https://pypi.org/project/JayDeBeApi/) library. Run the following command to install ibmdbpy, as well as the dependencies for the JDBC feature:
> pip install ibmdbpy[jdbc]
* Sample data: follow this [steps](https://www.ibm.com/support/knowledgecenter/SSCJDQ/com.ibm.swg.im.dashdb.doc/admin/sampledata.html ) to download sample data for Db2.  

__Association rules mining__

Association rules mining is a common technique to better know your customers' habits. It answers the question: what items are commonly bought together? As it enables you to more efficiently analyze the shopping baskets of your customers, it also gives you hints about which promotions or actions to propose. For example, if flour, sugar and couverture chocolate are frequently bought together, discounting couverture chocolate might boost the sales of flour and sugar, because they are the basic ingredients you need for a tasty cake with a nice chocolate topping. You can find more details about associations rules mining and its support in Db2 Warehouse in the [IBM Knowledge Center](https://www.ibm.com/support/knowledgecenter/SS6NHC/com.ibm.swg.im.dashdb.analytics.doc/doc/r_association_rules.html). 

__Contents__

This notebook uses the transactions dataset of an outdoor store. Let's see what patterns we find!

__1. Get started__
* Import what you need
* Establish a JDBC connection to Db2 Warehouse
* Explore the data
    
__2. Mine Association Rules with ibmdbpy__
* Reminder about Association Rules Mining
    * General task
    * Definitions
* Put your knowledge into practice
    * Define an AssociationRules object
    * Analyze the results
    * Experiment for yourself with pruning!

## 1. Get started

__Import what you need__

As usual, we first import some basic modules from ibmdbpy.

In [1]:
from ibmdbpy import IdaDataBase, IdaDataFrame

__Establish a JDBC connection to Db2 Warehouse__

Please enter your credentials in order to connect to the database.

In [2]:
#Enter the values for you database connection
dsn_database = "___" # e.g. "BLUDB"
dsn_hostname = "___" # e.g.: "abc.url.example"
dsn_port = "___"    # e.g. "50000"
dsn_uid = "___"     # e.g. "db2_1234"
dsn_pwd = "___"     # e.g. "zorglub"

In [3]:
#Establish connection
connection_string='jdbc:db2://'+dsn_hostname+':'+dsn_port+'/'+dsn_database+':user='+dsn_uid+';password='+dsn_pwd+";" 
idadb=IdaDataBase(dsn=connection_string, verbose = False)
# Set verbose to True if you want to see the detail of ibmdbpy operations

By default the environment variable `AUTOCOMMIT` is then set to True, which means that every SQL statement which is submitted through the connection is executed within its own transaction and then committed implicitly. When you close the connection to Db2, if the environment variable `AUTOCOMMIT` is set to False, then all changes after the last explicit commit are discarded. 

__Explore the data__

We use Db2 Warehouse sample data that can be downloaded [here](https://www.ibm.com/support/knowledgecenter/SSCJDQ/com.ibm.swg.im.dashdb.doc/admin/sampledata.html) . 

* GOSALES.ORDER_DETAILS contains data about transactions: product, price, quantity, shipment etc. 
* GOSALES.PRODUCT presents information about each product's characteristics: product type, color, brand etc. 
* GOSALES.PRODUCT_TYPE contains the name of each product type in various languages such as arabic, german, english, corean etc. 
* GOSALES.PRODUCT_NAME_LOOKUP contains, for each product, its name in all contries' language where it is sold. 

Let's define IdaDataFrames to explore our data.

* __Explore the original tables__

In [4]:
orders = IdaDataFrame(idadb, "GOSALES.ORDER_DETAILS")
orders.head()

,ORDER_DETAIL_CODE,ORDER_NUMBER,SHIP_DATE,PRODUCT_NUMBER,PROMOTION_CODE,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE
0,1000001,100001,2010-01-19 00:00:00,41110,0,256,15.62,35.09,33.69
1,1000002,100001,2010-02-17 00:00:00,69110,0,92,49.69,110.00,102.30
2,1000013,100002,2010-01-19 00:00:00,75110,0,162,80.00,119.69,111.31
3,1000014,100002,2010-01-19 00:00:00,76110,0,172,23.53,40.52,38.90
4,1000015,100002,2010-01-19 00:00:00,85110,0,74,176.47,359.60,334.43


In [5]:
o1 = orders['ORDER_NUMBER'].count_distinct()
print("Distinct transactions: %s" %o1)

Distinct transactions: 53256


We will apply the association rules mining algorithm to the `orders` IdaDataFrame : it contains both the transaction IDs (column `ORDER_NUMBER`) and the product IDs (column `PRODUCT_NUMBER`). The other tables are useful to get some additional or contextual information.

In [6]:
products = IdaDataFrame(idadb, "GOSALES.PRODUCT")
products.head()

,PRODUCT_NUMBER,BASE_PRODUCT_NUMBER,INTRODUCTION_DATE,DISCONTINUED_DATE,PRODUCT_TYPE_CODE,PRODUCT_COLOR_CODE,PRODUCT_SIZE_CODE,PRODUCT_BRAND_CODE,PRODUCT_IMAGE
0,1110,1,2001-02-15 00:00:00,None,951,908,808,701,P01CE1CG1.jpg
1,2110,2,2001-02-15 00:00:00,None,951,906,807,701,P02CE1CG1.jpg
2,3110,3,2001-02-15 00:00:00,None,951,924,825,701,P03CE1CG1.jpg
3,4110,4,2001-02-15 00:00:00,None,951,923,804,701,P04CE1CG1.jpg
4,5110,5,2001-02-15 00:00:00,None,951,923,823,701,P05CE1CG1.jpg


How many different products do we have?

In [7]:
n = products['PRODUCT_NUMBER'].count_distinct()
print("Distinct products: %s" %n)

Distinct products: 274


The only common column of ORDER_DETAILS and PRODUCT is `PRODUCT_NUMBER`.

The only common column of PRODUCT and PRODUCT_TYPE is `PRODUCT_TYPE_CODE`.

In [8]:
types = IdaDataFrame(idadb, "GOSALES.PRODUCT_TYPE")
types.head()

,PRODUCT_TYPE_CODE,PRODUCT_LINE_CODE,PRODUCT_TYPE_EN,PRODUCT_TYPE_AR,PRODUCT_TYPE_CS,PRODUCT_TYPE_DA,PRODUCT_TYPE_DE,PRODUCT_TYPE_EL,PRODUCT_TYPE_ES,PRODUCT_TYPE_FI,...,PRODUCT_TYPE_PL,PRODUCT_TYPE_PT,PRODUCT_TYPE_RO,PRODUCT_TYPE_RU,PRODUCT_TYPE_SC,PRODUCT_TYPE_SL,PRODUCT_TYPE_SV,PRODUCT_TYPE_TC,PRODUCT_TYPE_TH,PRODUCT_TYPE_TR
0,951,991,Cooking Gear,معدات الطهى,Potřeby pro vaření,Kogegrej,Kochzubehör,Εξοπλισμός μαγειρικής,Material de cocina,Keittovälineet,...,Sprzęt kuchenny,Material cozinha,Echipament pentru gătit,Кухонные принадлежности,炊事用具,Oprema za kuhanje,Köksredskap,炊具,อุปกรณ์ทำอาหาร,Mutfak aletleri
1,952,991,Tents,الخيام,Stany,Telte,Zelte,Σκηνές,Tiendas,Teltat,...,Namioty,Barracas,Corturi,Палатки,帐篷,Šotori,Tält,營帳,เต็นท์,Çadırlar
2,953,991,Sleeping Bags,المنامات,Spacáky,Soveposer,Schlafsäcke,Υπνόσακοι,Sacos de dormir,Makuupussit,...,Śpiwory,Saco de dormir,Saci de dormit,Спальные мешки,睡袋,Spalne vreče,Sovsäckar,睡袋,ถุงนอน,Uyku Tulumları
3,954,991,Packs,الحقائب,Batohy,Rygsække,Rucksäcke,Σακίδια,Mochilas,Reput,...,Plecaki,Mochilas,Pachete,Рюкзаки и сумки,背包,Nahrbtniki,Ryggsäckar,背包,เป้,Sırt Çantaları
4,955,991,Lanterns,الفوانيس,Svítilny,Lygter,Lampen,Λάμπες,Linternas,Lyhdyt,...,Latarki,Lanternas,Lanterne,Фонари,手提灯,Svetilke,Lampor,營燈,ตะเกียง,Fenerler


How many different product types do we have?

In [9]:
t1 = types['PRODUCT_TYPE_CODE'].count_distinct()
print('Distinct product types: %s' %t1)

Distinct product types: 21


PRODUCT_NAME_LOOKUP contains the names of the products in several languages as well as their description.

In [10]:
names = IdaDataFrame(idadb, "GOSALES.PRODUCT_NAME_LOOKUP")
names.head()

,PRODUCT_NUMBER,PRODUCT_LANGUAGE,PRODUCT_NAME,PRODUCT_DESCRIPTION
0,1110,AR,حقيبة مياه TrailChef,حقيبة حفيفة قابلة للطي لحمل السوائل بطريقة سهل...
1,1110,EL,Δοχείο υγρών Μαρμίτα Σεφ,"Ελαφρύ, πτυσσόμενο δοχείο για εύκολη μεταφορά ..."
2,1110,DE,TrailChef Wasserbeutel,"Leichter, zusammenfaltbarer Beutel zum einfach..."
3,1110,DA,Sahara Vandtaske,Sammentrykkelig letvægtstaske til væsker. Bred...
4,1110,CS,Vak na vodu Kuchtík,"Lehký, skladný vak na tekutiny. Široké hrdlo u..."


Here you see that products with different product numbers (IDs) may have the same product name in the same language, for example because there is just a variation of size or color. There are less product names and descriptions than product numbers.

In [11]:
names[names["PRODUCT_LANGUAGE"]=="EN"].tail(10)

,PRODUCT_NUMBER,PRODUCT_LANGUAGE,PRODUCT_NAME,PRODUCT_DESCRIPTION
264,151110,EN,Astro Pilot,This GPS has a high-contrast 86mm color screen...
265,151120,EN,Astro Pilot,This GPS has a high-contrast 86mm color screen...
266,151130,EN,Astro Pilot,This GPS has a high-contrast 86mm color screen...
267,152110,EN,Sky Pilot,"Small and lightweight, this GPS has a 55mm col..."
268,153110,EN,Auto Pilot,The Auto Pilot is an easy-to-use navigation sy...
269,154110,EN,Kodiak,"The Kodiak includes a tough leather strap, pre..."
270,154120,EN,Kodiak,"The Kodiak includes a tough leather strap, pre..."
271,154130,EN,Kodiak,"The Kodiak includes a tough leather strap, pre..."
272,154140,EN,Kodiak,"The Kodiak includes a tough leather strap, pre..."
273,154150,EN,Kodiak,"The Kodiak includes a tough leather strap, pre..."


In [12]:
names[names["PRODUCT_LANGUAGE"]=="EN"].count_distinct()

PRODUCT_NUMBER         274
PRODUCT_LANGUAGE         1
PRODUCT_NAME           144
PRODUCT_DESCRIPTION    141
dtype: int64

* __Create a view__


As you can see from the results of the last statements, the value of PRODUCT_NAME does not always identify a single product. In addition it is sometimes not clear from name of product to which type of product it belongs. So let’s create the view PRODUCT_NAME_TYPE_LOOKUP using the GOALES.PRODUCT, GOSALES.PRODUCT_NAME_LOOKUP and GOSALES.PRODUCT_TYPE table where the concatenation of PRODUCT_NAME, PRODUCT_NUMBER and PRODUCT_TYPE_NAME can be used as a unique and understandable name for a product. 

This new view will be useful to get the full English name of a product in order to interpret the rules.

In [13]:
# Create view PRODUCT_NAME_TYPE_LOOKUP

idadb.ida_query("create or replace view PRODUCT_NAME_TYPE_LOOKUP as select p.PRODUCT_NUMBER, PRODUCT_NAME, PRODUCT_TYPE_EN, PRODUCT_NAME || '-' || n.PRODUCT_NUMBER || '-' || PRODUCT_TYPE_EN as PRODUCT_NAME_TYPE from GOSALES.PRODUCT p, GOSALES.PRODUCT_NAME_LOOKUP n, GOSALES.PRODUCT_TYPE t where p.PRODUCT_NUMBER = n.PRODUCT_NUMBER and p.PRODUCT_TYPE_CODE = t.PRODUCT_TYPE_CODE and n.PRODUCT_LANGUAGE='EN';")


Let's take a quick look at it. We define an IdaDataFrame pointing at this view, with `PRODUCT_NUMBER` as indexer column.

In [14]:
# Define IdaDataFrame
idadf = IdaDataFrame(idadb, 'PRODUCT_NAME_TYPE_LOOKUP', indexer = 'PRODUCT_NUMBER')
idadf.head()

,PRODUCT_NUMBER,PRODUCT_NAME,PRODUCT_TYPE_EN,PRODUCT_NAME_TYPE
0,1110,TrailChef Water Bag,Cooking Gear,TrailChef Water Bag-1110-Cooking Gear
1,2110,TrailChef Canteen,Cooking Gear,TrailChef Canteen-2110-Cooking Gear
2,3110,TrailChef Kitchen Kit,Cooking Gear,TrailChef Kitchen Kit-3110-Cooking Gear
3,4110,TrailChef Cup,Cooking Gear,TrailChef Cup-4110-Cooking Gear
4,5110,TrailChef Cook Set,Cooking Gear,TrailChef Cook Set-5110-Cooking Gear


We will use this view for association rules interpretation. Instead of displaying rules as a complicated list of IDs, we want to be able to read them directly as "product A goes with product B", in plain language. 

But as seen before,  products with distinct IDs may have the same product name. So in order to be able to differentiate them we concatenate the name and the ID. Additionally we concatenate the product type to have a better understanding of the product at first glance. This is what you see in the `PRODUCT_NAME_TYPE` column.

In [15]:
#SANITY CHECK: we should obtainthe same number of distinct items
n1 = idadf['PRODUCT_NUMBER'].count_distinct()
n2 = idadf['PRODUCT_NAME_TYPE'].count_distinct()

print("Distinct products: %s" %n1)
print("Distinct product names: %s" %n2)

Distinct products: 274
Distinct product names: 274


Right, we now have a good overview of our datasets. Let's mine association rules!

## 2. Mine Association Rules with ibmdbpy

### Reminder about Association Rules Mining


__General task__

A collection of instances, here the content of shopping baskets, is our input. We want to output rules to predict the values of any attribute from the values of other attributes: e.g. knowing some items of a shopping basket, what other products are likely to be purchased?

* Definitions

**Support of an itemset in a transaction list**: fraction of transactions of the dataset containing this itemset.
> support(itemset) = number of transactions containing the itemset / total number of transactions in the dataset

For example, if your transaction dataset is {{bread, peanut butter}, {milk, bread, eggs, bacon}, {milk, bread, butter}}, then the support of the itemset {bread, milk} is 2/3.

**Frequent itemset**: an itemset whose support in your transaction list is greater than some threshold t.

**Rule R**: an implication of type "X--> Y". For example: {peanut butter} --> {bread}, {milk, bread, eggs} --> {bacon, butter}, {beer, crips} --> {pizza}

Note: "-->" means co-occurrence... not causality!

**Rule head** and **rule body**: The principal parts of an association rule are the rule body (also referred to as antecedent) and the rule head (also referred to as consequent). For example, where X --> Y is an association rule: The item set X is the rule body. The item Y is the rule head.

* Evaluation of association rules

Take the association rule R: "X-->Y" where X and Y are two itemsets.
Then the support of the rule R is defined by the fraction of transactions containing all the elements from both X and Y.
> support("X-->Y") = support(X ∪ Y)

The confidence of the rule is the fraction of itemsets containing X and Y relatively to the number of itemsets containing X. 
> confidence("X-->Y") = support("X-->Y") / support(X) = support(X ∪ Y) / support(X)

The lift of a rule can give a better assessment than its confidence because it considers both the confidence of the rule and the distribution of Y, hence the overall data set. It can be seen as the rule confidence divided by the support of the rule head Y.
> lift ("X-->Y") = confidence("X-->Y") / support(Y)


### Put your knowledge into practice

We use ibmdbpy library dedicated to machine learning. You can visit the documentation [here](https://pythonhosted.org/ibmdbpy/association_rules.html).

In [16]:
import ibmdbpy.learn.association_rules as assoc

__Define an AssociationRules object__

It is ready to be used for fitting and prediction:

In [17]:
Assoc = assoc.AssociationRules(modelname="SHOPPING_RULES", minsupport=0.04, maxlen=4, maxheadlen=1, minconf=0.2)

The AssociationRules class provides an interface for using the `ASSOCRULES` and `PREDICT_ASSOCRULES` IDAX methods of Db2.

* **minsupport**: The minimum fraction (0.0 - 1.0) or the minimum number (above 1) of transactions that must contain a pattern to be considered as frequent. By default, it ranges between >0.0 and <1.0 for a minimum fraction, or above 1 for a minimum number of transactions.

* **maxlen**: The maximum length of a pattern or a rule, that is, the maximum number of items per pattern or rule. Default is 5.

* **maxheadlen**: The maximum length of a rule head, that is, the maximum number of items that might belong to the item set on the right side of a rule. Increasing this value might significantly increase the number of detected rules.

* **minconf** : float, optional. It is the minimum confidence that a rule must achieve to be kept in the model of the pattern. >=0.0 and <= 1, default is 0.5. 

In [18]:
#Inspect the object
Assoc.get_params()

{'modelname': 'SHOPPING_RULES',
 'minsupport': 0.04,
 'maxlen': 4,
 'maxheadlen': 1,
 'minconf': 0.2,
 'nametable': None,
 'namecol': None,
 'outtable': None,
 'type': None,
 'limit': None,
 'sort': None}

You can see that many parameters, such as `nametable` and `namecol` have not been set yet. They can be set at fitting step. `nametable` is a table containing the explicit names of all products, with their IDs. We specify `namecol` to explicitly identify, in this table, the column containing the names.

In [19]:
Assoc.fit(orders, transaction_id="ORDER_NUMBER", item_id="PRODUCT_NUMBER", nametable = "PRODUCT_NAME_TYPE_LOOKUP", namecol='PRODUCT_NAME_TYPE', verbose=True)

assocpatterns
    ITEMSETID  ITEMID
0           1       1
1           2       2
2           3       3
3           4       4
4           5       5
5           6       6
6           7       7
7           8       8
8           9       9
9          10      10
10         11      11
11         12      12
12         13      13
13         14      14
14         15      15
15         16      16
16         17      17
17         18      18
18         19      19
19         20      20
20         21      21
21         22      22
22         23      23
23         24      24
24         25      25
25         26      26
26         27      27
27         28      28
28         29      29
29         30      30
..        ...     ...
46         47      47
47         48      48
48         49      49
49         50      50
50         51      51
51         52      52
52         53      53
53         54      54
54         57       5
55         62      19
56         64      38
57         65      38
58         55     

**Understand the intermediary tables**

In the cell above, intermediate tables have been printed because we have set `verbose` to True. You can also obtain them with `AssociationRules.describe(detail=True)`. Here is an explanation of these tables.

The first table **assocpatterns** shows which itemset (`ITEMSETID` column) contains which items (`ITEMID` column). Only the frequent itemsets are shown i.e. itemsets that are eligible according to the constraints you have set on your AssociationRules object. 

For example, the table above tells you that 65 itemsets are considered to be frequent. 

    1:{1}, 2:{2}, 3:{3}, ... 55:{4,38},..., 59:{12,38},..., 62:{19,41},..., 64:{42,38}, 65:{38, 51} 
    
Frequent itemset with itemsetid 1 contains product with itemid 1, frequent itemset with itemsetid 2 contains product with itemid 2, ..., frequent itemset with itemsetid 55 contains products with itemid 4 and 38, ... frequent itemset with itemsetid 65 contains products with itemid 38 and 51.
    
Note that the item IDs are the IDs of the items in table "item". They are not the original product IDs from the product table.

The second table **assocpatterns_stats** provides for each itemset some metrics: its cardinal (`LENGTH`), its absolute frequency in the dataset (`COUNT`), its support and its lift, as well as whether it was "pruned".

The third table **assocrules** gives you the rules that have been found, according to the constraints you have set. Here we have 21 rules. 

    * Rule 1: itemset 4 --> itemset 38 id est {4} --> {38}, corresponds to itemset 55: {4,38};
    * Rule 3: itemset 5 --> itemset 38 id est {5} --> {38}, corresponds to itemset 56: {5,38};
    * Rule 18: itemset 38 --> itemset 42 id est {38} --> {42}, corresponds to itemset 64: {38,42};
    * Rule 21: itemset 51 --> itemset 38 id est {51} --> {38}, corresponds to itemset 65: {38,51}.
    
The **items** table provides the mapping between new item IDs (`ITEMID`) and original item IDs (`ITEM`). It uses the `PRODUCT_NAME_TYPE_LOOKUP`table for interpretation and also gives support and count for each item.

__Analyze the results__

Printing all these intermediary tables with new IDs can be confusing... The `describe()` method is there to provide you with an overview of the rules, with product names for better understanding.

In [20]:
# You can print the tables again with the describe method:
Assoc.describe()

Summary of the rules
    ID                                               RULE   SUPPORT      LIFT  \
0   17     (Legend-145170-Watches) => (TX-144180-Watches)  0.078207  2.536326   
1   16     (TX-144180-Watches) => (Legend-145170-Watches)  0.078207  2.536326   
2   12   (Infinity-129130-Watches) => (TX-144180-Watches)  0.060951  2.415382   
3   13   (TX-144180-Watches) => (Infinity-129130-Watches)  0.060951  2.415382   
4    3      (Venue-125110-Watches) => (TX-144180-Watches)  0.054510  2.525859   
5    4      (TX-144180-Watches) => (Venue-125110-Watches)  0.054510  2.525859   
6    1    (Cat Eye-124190-Eyewear) => (TX-144180-Watches)  0.046887  2.133576   
7    2    (TX-144180-Watches) => (Cat Eye-124190-Eyewear)  0.046887  2.133576   
8    7      (Dante-126140-Eyewear) => (TX-144180-Watches)  0.046793  2.128928   
9    8      (TX-144180-Watches) => (Dante-126140-Eyewear)  0.046793  2.128928   
10   5  (Venue-125110-Watches) => (Legend-145170-Watches)  0.046624  2.976649   
11   6 

* Comment

It seems like in this outdoor store, clients often buy several watches or several pair of glasses at a time and also eyewear and watches together. For example if you buy a "Legend" watch you are likely to buy a "Venue" watch too. Same for "Fairway" eyewear  and "TX" watch, "Retro" eyewear  and "TX" watch etc. 

The best rule links the items "TX-144180-Watches" (support = 0.206118) and "Legend-145170-Watches" (support = 0.149598) and has the highest lift as well as the highest confidence. When a customer buys a Legend watch he is twice more likely to buy a TX watch as the others.

"Max Gizmo" knives are "frequent" because their support was above the threshold but this product does not appear in any of the rules above because no rule using this product could satisfy the confidence constraint we defined.

* Confidence interpretation

Don't be mislead by high confidence values! A rule with higher confidence could seem more accurate, because better supported. However we have to assess the accuracy of a rule relatively to the dataset. So a high confidence does not necessarily mean that there is a high level of implication in the rule. Sometimes it's just that Y is frequent (or much more frequent than X), so that support(X ∪ Y) / support(X) is high. That is why we use the lift metric.


* Lift interpretation 

Remember that the value of lift gives a better assessment because it considers both the confidence of the rule and the distribution of Y, hence the overall data set. 

If the lift is **equal to 1**, then the probability of occurrence of the antecedent X and that of the consequent Y are independent of each other. This means that **no rule** can be drawn involving these two events.
If the lift is **above 1**, then it tells us to which degree these two occurrences are **dependent on one another**, and we can try to use these rules to make prediction on future datasets.
If the lift is **under 1**, then the items are **substitute to each other**. This means that presence of one item has a negative effect on presence of the other item and vice versa. If you buy product X you are less likely to buy product Y and vice versa.

__Experiment for yourself with pruning__

Pruning enables you to remove rules and pattern which you are not interested in. Here we proceed by filtering the rules that have been selected above.

In [21]:
Assoc.prune(itemsin=None, itemsout=None, minlen=1, maxlen=None, 
           minsupport=0.06, maxsupport=1, minlift=None, maxlift=None, 
           minconf=None, maxconf=None, reset=True)

Assoc.describe()

Summary of the rules
   ID                                              RULE   SUPPORT      LIFT  \
0  17    (Legend-145170-Watches) => (TX-144180-Watches)  0.078207  2.536326   
1  16    (TX-144180-Watches) => (Legend-145170-Watches)  0.078207  2.536326   
2  12  (Infinity-129130-Watches) => (TX-144180-Watches)  0.060951  2.415382   
3  13  (TX-144180-Watches) => (Infinity-129130-Watches)  0.060951  2.415382   

   CONFIDENCE  
0    0.522781  
1    0.379430  
2    0.497853  
3    0.295709  


Note: when printing the intermediary table by setting the option `detail=True` in `describe()`, rules and patterns that have been removed from the model are tagged with a 1 in column `PRUNED`. As shown in the summary table above, only 4 rules remain with our current filtering constraints. Also note the symmetry of the lift and support metrics.

___
__Close the connection to Db2__

Congratulations! You reached the end of this notebook. Before closing the notebook, close the connection to Db2 with a simple line of code.

In [22]:
# Close the connection to IBM Db2 Warehouse
idadb.close()

Connection closed.


__Congratulations!__ You are ready to perform association rules mining with ibmdbpy!


## Where to go from here ?

More examples of ibmdbpy capabilities and ML algorithms are available on this GitHub repository.

* Getting started with ibmdbpy :
        
    [Basics](../GettingStarted/ibmdbpy_GettingStarted_1-basics.ipynb)

    [GeoBasics](../GettingStarted/ibmdbpy_GettingStarted_2-geo_basics.ipynb)

    [Extensive Guide](../GettingStarted/ibmdbpy_GettingStarted_3-geo_guide.ipynb)
    

* More practice : analyze the Museums dataset, understand how to create IdaDataFrames and IdaGeoDataFrames:
        
    [Preprocessing](../MuseumsUseCase/ibmdbpy_Museums_DataAnalysis_1-preprocessing.ipynb)

    [Geospatial recommendation](../MuseumsUseCase/ibmdbpy_Museums_DataAnalysis_2-geospatial.ipynb)


* Machine learning with ibmdbpy: 
        
    [Naïve Bayes](../MachineLearning/ibmdbpy_NaiveBayes.ipynb)

    [KMeans](../GettingStarted/ibmdbpy_GettingStarted_1-basics.ipynb) 

________
__Author__

Eva Feillet, ML intern @ IBM Research & Development Lab, Germany. IBM Cloud and Cognitive Software, Böblingen, Germany.